In [ ]:
# default_exp Voutput

## 基础信息：
**模块名称：Voutput
模块用途：将DataFrame\SimpleTable\Summary等格式数据输出为Latex文档或CSV文件
底层依赖库：pandas, numpy, statsmodels, os**

## 载入依赖模块：

**以下为Voutput模块所需：**

In [1]:
import pandas
import numpy
import statsmodels.api
import os

**以下为生成测试数据所需：**

In [2]:
import scipy.sparse as sparse
import statsmodels.api as sm
import statsmodels.stats.diagnostic
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import coint
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tsa.vector_ar.vecm import coint_johansen

***

## 生成测试数据：

In [10]:
raw_df= pandas.read_csv(
    "./data/df_japan.csv",
    header=None,
    encoding="gbk",
    names=["time", "none", "catagory", "title", "content"],)
time_column = pandas.to_datetime(raw_df['time']).dt.to_period("M").dt.to_timestamp()
lda_score = pandas.read_csv("./data/lda_7.csv")

In [11]:
new_col=["中日双边合作",
    "中美经贸带动世界经济发展",
    "中国社会发展",
    "中日企业合作",
    "中日历史与抗战认识问题",
    "世界和平和国际安全",
    "中国参加奥运会"]
mid_df = pandas.concat([time_column, lda_score], axis=1).groupby(['time']).agg('mean')
mid_df.columns=new_col
fin_df=mid_df.drop(["中国参加奥运会"],axis=1)

In [14]:
#ADF检验
t_list = []
name_list = []
p_list = []
for i in fin_df.columns:
    name_list.append(i)
    test_output = list(adfuller(fin_df.loc[:,i]))
    t_list.append(test_output[0])
    p_list.append(test_output[1])  
adf1_df=pandas.DataFrame({'name': name_list, 't_value': t_list, 'p_value': p_list})

In [15]:
#一阶差分后ADF
diff_fin_df=fin_df.drop(["中日历史与抗战认识问题"],axis=1).diff(periods=1).dropna()
t_list1 = []
name_list1 = []
p_list1 = []
for i in diff_fin_df.columns:
    name_list1.append(i)
    test_output = list(adfuller(diff_fin_df.loc[:, i]))
    t_list1.append(test_output[0])
    p_list1.append(test_output[1])
adf2_df=pandas.DataFrame({'name': name_list1, 't_value': t_list1, 'p_value': p_list1})

In [22]:
#export
#OLS相关
def ols(x,y):
    est=sm.OLS(y,sm.add_constant(x)).fit()
    return est.summary()

In [32]:
#VAR相关
var_df=fin_df.shift(periods=3).dropna()
var_mod = sm.tsa.VAR(var_df.loc["1946-08-01":]).fit( ic='aic')
ic=var_mod.model.select_order().summary()

c:\users\92859\appdata\local\programs\python\python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'


In [35]:
#irf相关
irf = var_mod.irf(24)# 24期

In [37]:
#vecm相关
vecm_mod=sm.tsa.VECM(var_df.loc["1946-08-01":]).fit()

c:\users\92859\appdata\local\programs\python\python38\lib\site-packages\statsmodels\tsa\base\tsa_model.py:524: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  warnings.warn('No frequency information was'


In [50]:
#fevd相关
fevd=var_mod.fevd(24)

***

## 模块内容：

#### 0. 公共函数 `Voutput.ispath_exist(path)`:
判断输出文件路径是否存在：

In [3]:
def ispath_exist(path):
    isExists=os.path.exists(path)
    if not isExists:
        os.makedirs(path)

***

 #### 1. （函数方法）`Voutput.df2csv(DataFrame, Filename)`:
**参数：** 
**DataFrame：** 输入数据应为DataFrame格式，可使用type()方法判断
**Filename：** 可以包含路径，不可包含后缀

**输出：** 单独的CSV文件

In [17]:
# 输入数据应为DataFrame格式
def df2csv(df,filename):    #通用函数，输出pd至csv
    ispath_exist('./output')
    df.to_csv(filename+'.csv',encoding='utf_8_sig')    #解决输出中文乱码

**演示**：

In [40]:
type(adf1_df)

pandas.core.frame.DataFrame

In [18]:
df2csv(adf1_df,'./output/original ADF')
df2csv(adf2_df,'./output/diff ADF')

***

 #### 2. （类方法）`Voutput.tbl2csv(Table, Filename)`:
**参数：** 
**Table：** 输入数据应为SimpleTable格式，可为以下几种：

- var_mod.model.select_order().summary() （VAR模型的信息准则）
- vecm_mod.summary().tables[i] （VECM回归报告特定表格）

**Filename：** 可以包含路径，不可包含后缀

**输出：** 单独的CSV文件


In [30]:
#输入数据应为SimpleTable格式，可为以下几种：
# var_mod.model.select_order().summary() （VAR模型的信息准则）
# vecm_mod.summary().tables[i] （VECM回归报告特定表格）
class tbl2csv():
    def __init__(self,table,filename):
        ispath_exist('./output')
        self.table=table
        self.filename=filename
        self.tbl2pd()

    def tbl2pd(self):
        tbl_list=list(self.table)
        df_tbl=pandas.DataFrame(tbl_list)
        df_tbl.to_csv(self.filename+'.csv',encoding='utf_8_sig',index=False, header=False)

**演示**：

In [41]:
type(ic)

statsmodels.iolib.table.SimpleTable

In [33]:
tbl2csv(ic,'./output/VAR information riterion')

***

#### 3. （函数方法）`Voutput.vecm2csv(Summary, Filename)`:
输出VECM模型的回归报告
**参数：** 
**Summary：** 输入数据应为Summary格式，可为：

- statsmodels.iolib.summary.Summary VECM回归报告

**Filename：** 可以包含路径，不可包含后缀

**输出：** 单独的CSV文件

In [39]:
# 输入数据应为Summary格式，可为：
# statsmodels.iolib.summary.Summary （VECM回归报告）
def vecm2csv(summary,filename):
    data=summary.as_csv()
    with open(filename+'.csv', 'a') as file_handle:
        file_handle.write(data)  # 写入
        file_handle.write('\n')

**演示**：

In [45]:
type(vecm_mod.summary())

statsmodels.iolib.summary.Summary

In [38]:
vecm2csv(vecm_mod.summary(),'./output/VECM')

***

 #### 4. （类方法）`vp.irf2csv(irf_obj, Foldname)`:
输出VAR模型脉冲响应图的数据点
**参数：** 
**irf_obj：** 目标对象为irf = var_mod.irf()，为以下数据类型：

- statsmodels.tsa.vector_ar.irf.IRAnalysis

**Foldname：** 由于输出多个文件，因此参数为文件夹名，写法和文件相同，最后不以/结尾

**输出：** 
两个CSV文件，一个为脉冲相应图数据点，另一个为相应的标准差

In [46]:
# 目标对象为irf = var_mod.irf()，为以下数据类型：
# statsmodels.tsa.vector_ar.irf.IRAnalysis
class irf2csv():  # 输出脉冲响应图数据
    def __init__(self, irf_obj, foldname):
        self.irf_obj = irf_obj
        self.foldname = foldname
        ispath_exist(self.foldname)
        self.get_names()
        self.irf_stderr()

    def irf_stderr(self):   #分别输出数据点和标准差
        self.irf_array=self.irf_obj.irfs
        datapoint=self.irf_array2df(self.irf_array)
        self.irf_df2csv(datapoint,'irf_datapoint')

        self.stderr_array=self.irf_obj.stderr()
        stderror = self.irf_array2df(self.irf_array)
        self.irf_df2csv(stderror, 'irf_stderror')

    def get_names(self):
        self.namelist=list()
        names=self.irf_obj.model.names  #变量名，查源码statsmodels/statsmodels/tsa/vector_ar/irf.py得到
        for i in names:
            temp1=str()
            temp1=i+'-->'
            for j in names:
                temp2=str()
                temp3=str()
                temp2=j
                temp3=temp1+temp2
                temp2=''
                self.namelist.append(temp3)

    def mergelist(self, array):  # 把多个array合并为一个list
        merged = list()
        for i in range(len(array)):
            for j in list(array[i]):  # 把ndarray转为list
                merged.append(j)
        return merged

    def irf_array2df(self,input_array):
        self.array=input_array
        graphic_list = list()
        dot_list = list()  # 按图像顺序记录最终点
        for i in self.array:
            graphic_list.append(i)
        graph_number = len(self.mergelist(graphic_list[0]))
        for index in range(graph_number):
            temp = list()
            for j in graphic_list:  # 获得一个图像的所有点
                temp.append(self.mergelist(j)[index])
            dot_list.append(temp)
        self.dot_df = pandas.DataFrame(dot_list,dtype=numpy.float)
        return self.dot_df

    def irf_df2csv(self,input_df,category):
        input_df.insert(0,'冲击顺序',self.namelist)
        input_df.to_csv(self.foldname+'/'+category+'.csv',encoding='utf_8_sig',index=False, header=True)  # 输出至csv

**演示**：

In [47]:
type(irf)

statsmodels.tsa.vector_ar.irf.IRAnalysis

In [48]:
irf2csv(irf,'./output/IRF')

***

 #### 5. （类方法）`vp.fevd2csv(fevd_obj, Foldname)`:
继承父类vp.irf2csv
输出VAR模型预测误差方差的数据
**参数：** 
**fevd_obj：** 目标对象为irf = var_mod.fevd()，为以下数据类型：

- statsmodels.tsa.vector_ar.var_model.FEVD （VAR模型预测误差方差）

**Foldname：** 由于输出多个文件，因此参数为文件夹名，写法和文件相同，最后不以/结尾

**输出：** 
多个CSV文件，每一个都以被解释变量结尾，内部为它与其他变量的方差误差数据

In [49]:
# 继承父类irf2csv
# 输入数据应为FEVD格式，可为：
# statsmodels.tsa.vector_ar.var_model.FEVD （VAR模型预测误差方差）
class fevd2csv(irf2csv):
    def __init__(self, fevd_obj, foldname):
        self.fevd_obj = fevd_obj
        self.foldname = foldname
        ispath_exist(self.foldname)
        self.get_names()
        self.fevd_array2df()

    def get_names(self):
        self.namelist=self.fevd_obj.names

    def fevd_df2csv(self,df,name):
        isExists = os.path.exists(self.foldname)
        if not isExists:
            os.makedirs(self.foldname)
        df.to_csv(self.foldname+'/'+name+'.csv',encoding='utf_8_sig',index=True, header=True)  # 输出至csv

    def fevd_array2df(self):
        fevd_arrays=self.fevd_obj.decomp
        for i in range(len(fevd_arrays)):
            name=self.namelist[i]   #被解释变量名称
            array=fevd_arrays[i]    #单张图数据量：期数x变量数
            dec_df = pandas.DataFrame(array)
            dec_df.columns=self.namelist
            self.fevd_df2csv(dec_df,name)

**演示**：

In [51]:
type(fevd)

statsmodels.tsa.vector_ar.var_model.FEVD

In [53]:
fevd2csv(fevd,'./output/FEVD dataset')

***

 #### 6. （函数方法）`vp.ols2latex(Summary, Foldname)`:
输出OLS模型的回归报告（Latex格式）
**参数：** 
**Summary：** 输入数据应为Summary格式，可为：

- statsmodels.iolib.summary.Summary OLS回归报告

**Foldname：** 对于多次OLS回归，生成多份OLS回归报告于同一个文件夹，最后不以/结尾

**输出：** 多份TXT文档


In [20]:
# 输入数据应为Summary格式，可为：
# statsmodels.iolib.summary.Summary （OLS/VECM回归报告）
def ols2latex(summary,foldname):
    fold=foldname
    ispath_exist(fold)
    filename=pandas.DataFrame(summary.tables[0])[1][0]
    data=summary.as_latex()
    with open(fold+'/'+str(filename)+'.txt', 'a') as file_handle:
        file_handle.write(data)  # 写入
        file_handle.write('\n')

**演示：**

In [27]:
def olsre(df):
    olsr=list()
    for i in df.columns:
        y=df[i]
        x=df.drop([i],axis=1)
        ols2latex(ols(x,y),'./output/OLS')
    return olsr

In [28]:
diff_fin_df["中日历史与抗战认识问题"]=fin_df.iloc[-1,4]

In [29]:
olsre(diff_fin_df)

[]

***

#### 7. （函数方法）`vp.vecm2latex(Summary, Filename)`:
输出VECM模型的回归报告（Latex格式）
**参数：** 
**Summary：** 输入数据应为Summary格式，可为：

- statsmodels.iolib.summary.Summary VECM回归报告

**Filename：** 可以包含路径，不可包含后缀

**输出：** 单独的TXT文件


In [54]:
def vecm2latex(summary,filename):
    data=summary.as_latex()
    ispath_exist('./output')
    with open(filename+'.txt', 'a') as file_handle:
        file_handle.write(data)  # 写入
        file_handle.write('\n')

**演示：**

In [45]:
type(vecm_mod.summary())

statsmodels.iolib.summary.Summary

In [56]:
vecm2latex(vecm_mod.summary(),'./output/VECM')